In [2]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import base64

In [3]:
#https://free-proxy-list.net/
"""
proxy中，有可用與不可用的，成功率約20-25%，會有一大串無法成功的proxy
"""

def get_proxy():
    a=requests.get('https://free-proxy-list.net/',headers={
        'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/536.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Mobile Safari/537.36'
    },timeout=20)
    soup=BeautifulSoup(a.text,'lxml')

    col_name=[s.text for s in soup.find(class_='table-responsive').find('tr').find_all('th')]

    d=[]

    for row in soup.find(class_='table-responsive').find_all('tr')[1:]:
        d.append([col.text for col in row])

    proxy_df=pd.DataFrame(d,columns=col_name)
    
    proxy_df=proxy_df[proxy_df.Anonymity.isin(['elite proxy','transparent'])]
    
    return proxy_df

In [3]:
#https://geonode.com/free-proxy-list/ 
"""
無任一proxy成功，表現非常差勁，不考慮
"""

def get_proxy_2():
    b=requests.get('https://proxylist.geonode.com/api/proxy-list?limit=200&sort_by=lastChecked&sort_type=desc').json()
    
    b=pd.DataFrame(b['data'])
    
    b=b.loc[:,['ip','port','country','org','asn','protocols','anonymityLevel','speed','upTime','responseTime','latency']]
    
    b=b[b['speed']<1000]
    
    b=b.rename(columns={
        'ip':'IP Address',
        'port':'Port'
    })
    
    return b

In [4]:
#https://hidemy.name/en/proxy-list/
"""
無任一proxy成功，表現非常差勁，不考慮
"""

def get_proxy_3():
    a=requests.get('https://hidemy.name/en/proxy-list/',headers={
        'referer': 'https://www.google.com/',
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    })
    soup=BeautifulSoup(a.text,'lxml')    
    
    u=['https://hidemy.name'+s.get('href') for s in soup.find(class_='pagination').find_all('a') if s.get('href') is not None]
    
    u.remove('https://hidemy.name/en/proxy-list/#list')
    
    data=[]
    
    data.append(pd.read_html(str(soup.find(class_='table_block')))[0])
    
    for url in u:
        a=requests.get(url,headers={
            'referer': 'https://www.google.com/',
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
        })
        soup=BeautifulSoup(a.text,'lxml')      
        
        data.append(pd.read_html(str(soup.find(class_='table_block')))[0])
    
    data=pd.concat(data)
    
    data=data.rename(columns={'IP address':'IP Address'})
    
    return data[data.Anonymity != 'no']

In [5]:
#https://www.proxyscan.io/
"""
非常微小IP可成功(0.5%成功率)
"""

def get_proxy_4():
    #選500筆
    a=requests.get('https://www.proxyscan.io/home/filterresult?limit=500')
    soup=BeautifulSoup(a.text,'lxml')

    row=[]

    for r in soup.find_all('tr'):
        row.append([s.text.strip() for s in r.find_all(re.compile('td|th'))])

    data=pd.DataFrame(row,columns=[
        'IP Address','Port','Country,City','Ping','Type','Anonymity','Uptime','Last Status','Last Check'
    ])

    data=data[data['Last Status'] == 'OK']
    
    return data

## Test free proxy website can use or not

In [6]:
for fun in [get_proxy,get_proxy_2,get_proxy_3,get_proxy_4]:
    proxy_df=fun()

    proxy_list=[{'https':f'{i}:{j}','http':f'{i}:{j}'} for i,j in zip(proxy_df['IP Address'],proxy_df['Port'])] 
    
    url='https://ip.xxoo.net/?L=tw'
    
    print(f'Now test function is {fun}')
    
    N=0

    for now_proxy in proxy_list:
        try:
            a=requests.get(url,timeout=20,proxies=now_proxy)
            soup=BeautifulSoup(a.text,'lxml') 
            N=N+1
        except Exception as e:
            continue

    total_n=len(proxy_list)
    print(f'Total test is {total_n} , success is {N}.Percentage of success is {np.round(N/total_n,4)*100}')

Now test function is <function get_proxy at 0x0000019487A9A9D8>
Total test is 211 , success is 61.Percentage of success is 28.910000000000004
Now test function is <function get_proxy_2 at 0x00000194CA329A68>
Total test is 111 , success is 0.Percentage of success is 0.0
Now test function is <function get_proxy_3 at 0x00000194CA329F78>
Total test is 37 , success is 0.Percentage of success is 0.0
Now test function is <function get_proxy_4 at 0x0000019487935948>
Total test is 107 , success is 0.Percentage of success is 0.0


In [5]:
proxy_df

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
1,81.24.117.250,18080,RU,Russian Federation,elite proxy,,no,1 min ago
2,54.36.176.76,1080,FR,France,elite proxy,,no,1 min ago
3,45.79.230.234,80,US,United States,elite proxy,no,yes,1 min ago
4,139.255.129.161,8080,ID,Indonesia,elite proxy,,no,1 min ago
5,93.99.12.253,8080,CZ,Czech Republic,elite proxy,,no,1 min ago
...,...,...,...,...,...,...,...,...
293,188.190.42.233,55277,UA,Ukraine,elite proxy,no,yes,32 mins ago
294,162.55.188.174,80,DE,Germany,elite proxy,,no,32 mins ago
295,128.1.61.222,808,SG,Singapore,elite proxy,no,yes,32 mins ago
296,175.196.182.39,80,KR,Korea,elite proxy,,no,32 mins ago
